In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl


df=pd.read_excel('patents_data.xlsx')


In [9]:
df.head()

,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary
0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,20200626,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,"본 발명의 실시 예에 따른 시스템은, 자동화 폐기물 수거함의 딥러닝 기반 인공지능 ..."
1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,20170807,강준철,이영규,강소라,NaN,NaN,"본 발명은, 인공지능(AI)에 관한 기술과 온라인 문자열 사전에 관한 기술을 활용하..."
2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,20220203,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,"본 발명은 품질검사를 위한 장치 및 방법에 관한 것으로, 본 발명에 따르면, 종래,..."
3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,20210812,조국환,NaN,NaN,NaN,NaN,본 발명은 광범위하게 활용되는 인터넷기반 부동산 정보를 때와 장소를 불문하고 기록하...
4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,20230316,주식회사 오콜로르,박상열,NaN,NaN,NaN,본 발명은 PI(Personal Identity) 정보 빅데이터를 구축하는 통하여 ...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364212 entries, 0 to 364211
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   status                     364212 non-null  object 
 1   title                      364212 non-null  object 
 2   ap_num                     364212 non-null  int64  
 3   application_date           364212 non-null  int64  
 4   applicant                  364212 non-null  object 
 5   representative             359245 non-null  object 
 6   final_owner                196016 non-null  object 
 7   priority_date              147903 non-null  float64
 8   international_filing_date  101183 non-null  float64
 9   summary                    364206 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 27.8+ MB


In [11]:
# categories = {
#     '제너럴': r'\b(추론|알고리즘|학습|훈련방법||훈련 방법|지도방법|지도 방법|학습방법|학습 방법|딥러닝|딥 러닝|)\b',
#     '의료': r'\b(의료|헬스|치료|건강|유전자|병원|환자)\b',
#     '전자상거래': r'\b(상거래|상품|배송|판매|결제|공급망|판매망)\b',
#     '자동차': r'\b(자동차|주행|자율주행|도로|차량|교통|보행자|차선|신호등)\b',
#     '금융': r'\b(금융|코인|트레이딩|투자|신용|금융사기|자산)\b',
#     '교육': r'\b(교육|강의|교육용|시험|성적|학생)\b',
#     '농업': r'\b(농업|스마트농업|농작물|작물|농약|수확|스마트팜)\b',
#     '엔터테인먼트': r'\b(콘텐츠생성|게임|미디어|사용자인터페이스|가상현실|음성인식)\b',
#     '보안': r'\b(침입|탐지|데이터보호|위협예측|인증시스템|보안|암호화)\b'
# }

In [12]:
# !pip install konlpy

In [6]:
# applicant 컬럼의 값 전처리 (공백 제거, 소문자 변환)
df['applicant_processed'] = df['applicant'].str.replace(' ', '').str.lower()

# 소그룹 및 대그룹 정의
small_grp= {
    '현대': ['현대', 'hyndai', '에이치디'],
    '기아': ['기아', 'kia'],
    '삼성': ['삼성', 'samsung'],
    '엘지': ['엘지', 'lg'],
    '한화': ['한화', 'hanhwa'],
    '엘에스': ['엘에스', 'ls'],
    '서울바이오시스': ['서울바이오시스'],
    '린나이코리아': ['린나이코리아'],
    '한국생산기술연구원': ['한국생산기술연구원', 'kitech'],
    '한국과학기술연구원': [ 'kist'], 
    '한국기계연구원': ['한국기계연구원', 'kimm'],
    '한국에너지기술연구원': ['한국에너지기술연구원', 'kier'],
    '한국건설기술연구원': ['한국건설기술연구원', 'kict'], 
    '한국해양과학기술원': ['한국해양과학기술원', 'kiost'],
    '한국전자통신 연구원': ['한국전자통신연구원', 'etri'], 
    '한국세라믹기술원': ['한국세라믹기술원', 'kicet'], 
    '한국지질 자원 연구원': ['한국지질자원연구원', 'kigam'],
    '가톨릭대학교': ['가톨릭대학교'],
    '서울대학교': ['서울대학교'],
    '고려대학교': ['고려대학교'],
    '연세대학교': ['연세대학교'],
    '한국과학기술원':['한국과학기술원','카이스트','kaist'],
    '포항공과대학교':['포항공과대학교','포스텍','postech'],
    '한양대학교':['한양대학교'],
    '경희대학교':['경희대학교'],
    '성균관대학교':['성균관대학교'],
    '타이완세미컨덕터': ['타이완세미컨덕터', 'taiwansemiconductor','tsmc'], 
    '캐논': ['캐논','canon'], 
    '애플': ['애플','apple'],  
    '화웨이': ['후아웨이','화웨이','huawei'],  
    '인텔': ['인텔','intel'],   
    '어플라이드머티어리얼스': ['어플라이드머티어리얼스','어플라이드머티리얼스', 'appliedmaterials'],
    '도요타': ['도요다','도요타','토요타','toyota'],   
    '한도오따이에네루기': ['한도오따이'],
    '고베세이코쇼': ['고베세이코쇼'], 
    '도쿄엘렉트론': ['도쿄엘렉트론','도쿄일렉트론','tokyoelectron'], 
    '퀄컴': ['퀄컴','qualcomm'],   
    '마이크론': ['마이크론','micron'],    
    '도레이': ['도레이','toray'], 
    '후지': ['후지','fuji'],  
    '도시바': ['도시바','toshiba'],
    '닛토덴코': ['닛토덴코'],
    '케이티': ['케이티', 'kt'],
    '스냅인코포레이티드': ['스냅'],
    '구글': ['구글'],
    '다이니폰인사츠': ['다이니폰인사츠'],
    '쌩-고벵글래스프랑스': ['쌩-고벵글래스프랑스'],
    '제이에프이스틸': ['제이에프이스틸'],
    '한국한의학연구원': ['한국한의학연구원'],
    '아사히가세이': ['아사히가세이'],
    '코쿠사이엘렉트릭': ['코쿠사이엘렉트릭'],
    '한국전자기술연구원': ['한국전자기술연구원'],
    '텐센트': ['텐센트'],
    '한국공학대학교': ['한국공학대학교'],
    '바이두': ['바이두'],
    '다이킨고교': ['다이킨고교'],
    '솔브레인': ['솔브레인'],
    '한국로봇융합연구원': ['한국로봇융합연구원'],
    '보에테크놀로지': ['보에테크놀로지'],
    '광동오포': ['광동오포'],
    '카와사키': ['카와사키'],
    '텔레호낙티에볼라게트': ['텔레호낙티에볼라게트'],
    '한국전력공사': ['한국전력공사'],
    '에이알엠': ['에이알엠'],
    '로베르트보쉬게엠베하': ['로베르트보쉬게엠베하'],
    '다이셀': ['다이셀'],
    '신에쓰가가꾸고교': ['신에쓰가가꾸고교'],
    '전북대학교': ['전북대학교'],
    '소니': ['소니'],
    '다이후쿠': ['다이후쿠'],
    '스미또모가가꾸': ['스미또모가가꾸'],
    '포스코': ['포스코','posco'],
    '한국재료연구원': ['한국재료연구원'],
    '국방과학연구소': ['국방과학연구소'],
    '한국과학기술정보연구원': ['한국과학기술정보연구원'],
    '아모레퍼시픽': ['아모레퍼시픽'],
    '메르크파텐트게엠베하': ['메르크파텐트게엠베하'],
    '바스프에스이': ['바스프에스이'],
    '지티이코포레이션': ['지티이코포레이션'],
    '세종대학교': ['세종대학교'],
    '레르리키드쏘시에떼아노님뿌르레뜌드에렉스뿔라따시옹데프로세데조르즈클로드': ['레르리키드쏘시에떼아노님뿌르레뜌드에렉스뿔라따시옹데프로세데조르즈클로드'],
    '닛폰세이테츠': ['닛폰세이테츠'],
    '서울반도체': ['서울반도체'],
    '다우글로벌테크놀로지스': ['다우글로벌테크놀로지스'],
    '미쓰비시': ['미쓰비시','미쯔비시'],
    '코니카미놀타': ['코니카미놀타'],
    '네이버': ['네이버'],
    '중앙대학교': ['중앙대학교'],
    '에이에스엠엘': ['에이에스엠엘', 'asml'],
    '코닝': ['코닝'],
    '한국단자공업': ['한국단자공업'],
    '인하대학교': ['인하대학교'],
    '한국화학연구원': ['한국화학연구원'],
    '비와이디': ['비와이디'],
    '한국소재융합연구원': ['한국소재융합연구원'],
    '샤오미': ['시아오미','샤오미'],
    '한국철도공사': ['한국철도공사'],
    '신에쯔한도타이': ['신에쯔한도타이'],
    '세키스이가가쿠고교': ['세키스이가가쿠고교'],
    '가부시끼가이샤레조낙': ['가부시끼가이샤레조낙'],
    '지멘스': ['지멘스'],
    '에스케이텔레콤': ['에스케이텔레콤'],
    '로레알': ['로레알'],
    '가부시기가이샤': ['가부시기가이샤'],
    '마이크로소프트': ['마이크로소프트'],
    '울산과학기술원': ['울산과학기술원'],
    '한국정보통신': ['한국정보통신'],
    '임머숀': ['임머숀'],
    '경북대학교': ['경북대학교'],
    '닛산가가쿠': ['닛산가가쿠'],
    '농촌진흥청': ['농촌진흥청'],
    '램리써치': ['램리써치'],
    '닛뽄가야쿠': ['닛뽄가야쿠'],
    '광운대학교': ['광운대학교'],
    '베이징바이트댄스네트워크': ['베이징바이트댄스네트워크'],
    '에이지씨': ['에이지씨'],
    '린텍': ['린텍'],
    '쿠팡': ['쿠팡'],
    '한국원자력연구원': ['한국원자력연구원'],
    '오스템임플란트': ['오스템'],
    '가천대학교': ['가천대학교'],
    '배재대학교': ['배재대학교'],
    '웨이모': ['웨이모'],
    '에스케이하이닉스': ['에스케이하이닉스', '하이닉스'],
    '레이스트라티직.': ['레이스트라티직'],
    '미츠비시': ['미츠비시'],
    '쓰리엠': ['쓰리엠'],
    '샤프': ['샤프'],
    '국민대학교': ['국민대학교'],
    '무라타': ['무라타'],
    '알까뗄루슨트': ['알까뗄루슨트'],
    '히다치': ['히다치'],
    '루미리즈': ['루미리즈'],
    '전남대학교': ['전남대학교'],
    '한국원자력의학원': ['한국원자력의학원'],
    '다이요유덴': ['다이요유덴'],
    '컨템포러리엠퍼렉스테크놀로지씨오.': ['컨템포러리엠퍼렉스테크놀로지씨오.'],
    '에스엠시': ['에스엠시'],
    '샌디스크': ['샌디스크'],
    '부산대학교': ['부산대학교'],
    '추가이세이야쿠': ['추가이세이야쿠'],
    '한국산업기술원': ['한국산업기술원'],
    '콘비다와이어리스': ['콘비다와이어리스'],
    '앰코': ['앰코'],
    '건국대학교': ['건국대학교'],
    '스카이워크스': ['스카이워크스'],
    '광주과학기술원': ['광주과학기술원'],
    '엔피': ['엔피'],
    '다이이찌산쿄': ['다이이찌산쿄'],
    '우시오덴키': ['우시오덴키'],
    '에프.호프만-라로슈아게': ['에프.호프만-라로슈아게'],
    '도오꾜오까고오교': ['도오꾜오까고오교'],
    '아주대학교': ['아주대학교'],
    '대구경북과학기술원': ['대구경북과학기술원'],
    '충북대학교': ['충북대학교'],
    '충남대학교': ['충남대학교'],
    '사빅글로벌': ['사빅글로벌'],
    '크루셜텍': ['크루셜텍'],
    '비즈모델라인': ['비즈모델라인'],
    '에어프로덕츠앤드케미칼스': ['에어프로덕츠앤드케미칼스'],
    '쯔바기모도체인': ['쯔바기모도체인'],
    '루미레즈': ['루미레즈'],
    '케이엘에이': ['케이엘에이'],
    '동국대학교': ['동국대학교'],
    '베리안': ['베리안'],
    '스크린': ['스크린'],
    '고등기술연구원': ['고등기술연구원'],
    '금호': ['금호'],
    '한국인터넷진흥원': ['한국인터넷진흥원'],
    '유니크': ['유니크'],
    '인터디지털브이씨': ['인터디지털브이씨'],
    '에이치엘만도': ['에이치엘만도', '한라', '만도'],
    '한국기술교육대학교': ['한국기술교육대학교'],
    '정하익': ['정하익'],
    '테슬라': ['테슬라'],
    '사우디아라비안오일': ['사우디아라비안오일'],
    '노파르티스아게': ['노파르티스아게'],
    '옴야인터내셔널아게': ['옴야인터내셔널아게'],
    '군산대학교': ['군산대학교'],
    '베이징다지아인터넷인포메이션': ['베이징다지아인터넷인포메이션'],
    '서울과학기술대학교': ['서울과학기술대학교'],
    '숭실대학교': ['숭실대학교'],
    '서강대학교': ['서강대학교'],
    '바디프랜드': ['바디프랜드'],
    '한국공항공사': ['한국공항공사'],
    '톰슨라이센싱': ['톰슨라이센싱'],
    '인터디지털씨이페이튼트에스에이에스': ['인터디지털씨이페이튼트에스에이에스'],
    '오씨아이': ['오씨아이'],
    '브리스톨-마이어스스큅': ['브리스톨-마이어스스큅'],
    '에이에스엠아이피': ['에이에스엠아이피'],
    '한국식품연구원': ['한국식품연구원'],
    '인제대학교': ['인제대학교'],
    '한국철도기술연구원': ['한국철도기술연구원'],
    '인천대학교': ['인천대학교'],
    '강원대학교': ['강원대학교'],
    '페트로리움앤드케미컬': ['페트로리움앤드케미컬'],
    '엔제루구루푸': ['엔제루구루푸'],
    '인터디지탈': ['인터디지탈'],
    '유니버셜디스플레이': ['유니버셜디스플레이'],
    '넥슨': ['넥슨'],
    '아폴로인텔리전트커넥티비티': ['아폴로인텔리전트커넥티비티'],
    '엔체인': ['엔체인'],
    '우리은행': ['우리은행'],
    '니프코': ['니프코'],
    '생-고뱅퍼포먼스플라스틱스': ['생-고뱅퍼포먼스플라스틱스'],
    '이화여자대학교': ['이화여자대학교'],
    '덴소': ['덴소'],
    '쟈트코': ['쟈트코'],
    '제이에스알가부시끼가이샤': ['제이에스알'],
    '오므론': ['오므론'],
    '돌비': ['돌비'],
    '인피니온': ['인피니온'],
    '제이엑스': ['제이엑스'],
    '일루미나인드': ['일루미나인'],
    '아서스': ['아서스'],
    '히타치': ['히타치'],
    '지앙수헨그루이파마슈티컬스': ['지앙수헨그루이파마슈티컬스'],
    '명화공업': ['명화공업'],
    '상하이마이크로일렉트로닉스이큅먼트': ['상하이마이크로일렉트로닉스이큅먼트'],
    '쿠라레': ['쿠라레'],
    '에바라': ['에바라'],
    '후루카와덴키고교': ['후루카와덴키고교'],
    '다이오페이퍼': ['다이오페이퍼'],
    '어플라이드메디컬리소시스': ['어플라이드메디컬리소시스'],
    '모토로라': ['모토로라'],
    '제넨테크': ['제넨테크'],
    '닛테츠스테인레스': ['닛테츠스테인레스'],
    '금오공과대학교': ['금오공과대학교'],
    '덴카': ['덴카'],
    '코어포토닉스': ['코어포토닉스'],
    '한림대학교': ['한림대학교'],
    '야스카와덴키': ['야스카와덴키'],
    '듀폰': ['듀폰'],
    '에자이': ['에자이'],
    '아마존': ['아마존'],
    '레이아': ['레이아'],
    '다이요': ['다이요'],
    '타이코': ['타이코'],
    '엔지케이인슐레이터': ['엔지케이인슐레이터'],
    '디아이씨': ['디아이씨'],
    '탑엔지니어링': ['탑엔지니어링'],
    '카이스': ['카이스'],
    '이베이': ['이베이'],
    '코나미데지타루': ['코나미데지타루'],
    '에코랍유에스에이': ['에코랍유에스에이'],
    '한전케이디엔': ['한전케이디엔'],
    '인피닉': ['인피닉'],
    '매직립': ['매직립'],
    '니폰제온': ['니폰제온'],
    '셀바스에이아이': ['셀바스에이아이'],
    '이녹스첨단소재': ['이녹스첨단소재'],
    '다탕모바일': ['다탕모바일'],
    '동서대학교': ['동서대학교'],
    '아이비엠': ['인터내셔널비지네스머신즈', 'ibm'],
    '야마하': ['야마하'],
    '르노': ['르노'],
    '인포뱅크': ['인포뱅크'],
    '다이요잉키세이조': ['다이요잉키세이조'],
    '다이헨': ['다이헨'],
    '카티바': ['카티바'],
    '에이젠글로벌': ['에이젠글로벌'],
    '아지노모토': ['아지노모토'],
    '영남대학교': ['영남대학교'],
    '부경대학교': ['부경대학교'],
    '드림텍': ['드림텍'],
    '아틀라스캅코에어파워남로체벤누트삽': ['아틀라스캅코에어파워남로체벤누트삽'],
    '계명대학교': ['계명대학교'],
    '닛폰쇼쿠바이': ['닛폰쇼쿠바이'],
    '호야렌즈타일랜드': ['호야렌즈타일랜드'],
    '동의대학교': ['동의대학교'],
    '스미또모가가꾸': ['스미또모가가꾸'],
    '도쿄오카고교': ['도쿄오카고교'],
    '아미코스메틱': ['아미코스메틱'],
    '베이징센스타임': ['베이징센스타임'],
    '한국전기연구원': ['한국전기연구원'],
    '코르테바애그리사이언스': ['코르테바애그리사이언스'],
    '어드밴스드마이크로디바이시즈': ['어드밴스드마이크로디바이시즈', 'amd'],
    '알박': ['알박'],
    '칭다오하이어워싱머신': ['칭다오하이어워싱머신'],
    '한국핵융합에너지연구원': ['한국핵융합에너지연구원'],
    '가부시끼가이샤구보다': ['가부시끼가이샤구보다'],
    '아르끄마프랑스': ['아르끄마프랑스'],
    '코나아이': ['코나아이'],
    '울산대학교': ['울산대학교'],
    '닛산지도우샤': ['닛산지도우샤'],
    '제네럴일렉트릭': ['제네럴일렉트릭'],
    '니혼마이크로닉스': ['니혼마이크로닉스'],
    '노키아테크놀로지스': ['노키아테크놀로지스'],
    '엑시스에이비': ['엑시스에이비'],
    '상하이센스타임인텔리전트테크놀로지': ['상하이센스타임인텔리전트테크놀로지'],
    '다우실리콘즈': ['다우실리콘즈'],
    '닛테츠케미컬앤드머티리얼': ['닛테츠케미컬앤드머티리얼'],
    '뷰노': ['뷰노'],
    '극동대학교': ['극동대학교'],
    '서울시립대학교': ['서울시립대학교'],
    '비씨카드': ['비씨카드'],
    '엘아이지넥스원': ['엘아이지넥스원'],
    '단국대학교': ['단국대학교'],
    '아데카': ['아데카'],
    '스미토모덴키고교': ['스미토모덴키고교'],
    '아익스트론에스이': ['아익스트론에스이'],
    '도판': ['도판'],
    '하마마츠포토닉스': ['하마마츠포토닉스'],
    '자이냅스': ['자이냅스'],
    '에보닉': ['에보니크오퍼레이션즈'],
    '지에스유아사': ['지에스유아사'],
    '프랙스에어테크놀로지': ['프랙스에어테크놀로지'],
    '데쿠세리아루즈': ['데쿠세리아루즈'],
    '알리바바': ['알리바바'],
    '가즈트랑스포르에떼끄니가즈': ['가즈트랑스포르에떼끄니가즈'],
    '크라우드웍스': ['크라우드웍스'],
    '딥브레인에이아이': ['딥브레인에이아이'],
    '에어버스헬리콥터스': ['에어버스헬리콥터스'],
    '경상국립대학교': ['경상국립대학교'],
    '에임스': ['에임스'],
    '고마쓰세이사쿠쇼': ['고마쓰세이사쿠쇼']
}

large_grp = {
    '국내대기업': ['현대', 'hyndai', '에이치디', '기아', 'kia', '삼성', 'samsung', '엘지', 'lg', '한화', 'hanhwa', '엘에스',
        'ls', '서울바이오시스', '린나이코리아', '케이티', 'kt', '포스코', 'posco','아모레퍼시픽','네이버','에스케이텔레콤',
	    '에스케이하이닉스', '하이닉스','금호','에이치엘만도','우리은행','비씨카드'],
    '국내기업': ['솔브레인','지티이코포레이션','서울반도체','한국단자공업','엔피','크루셜텍','비즈모델라인',
	    '유니크','바디프랜드','오씨아이','에이에스엠아이피','넥슨','탑엔지니어링','카이스','한전케이디엔',
	    '인피닉','셀바스에이아이','이녹스첨단소재','명화공업','인포뱅크','카티바','에이젠글로벌','아미코스메틱',
	    '코나아이','뷰노','엘아이지넥스원','자이냅스','크라우드웍스','딥브레인에이아이','에임스'],
    '국내연구소': ['한국생산기술연구소', 'kitech', '한국과학기술연구소', 'kist', '한국기계연구원', 'kimm',
        '한국에너지기술연구원', 'kier', '한국건설기술연구원', 'kict', '한국해양과학기술원', 'kiost',
        '한국전자통신 연구원', 'etri', '한국세라믹기술원', 'kicet', '한국지질 자원 연구원', 'kigam',
	    '한국한의학연구원', '한국전자기술연구원','한국로봇융합연구원','한국전력공사',
	    '한국재료연구원','국방과학연구소','한국과학기술정보연구원','한국화학연구원','한국소재융합연구원',
	    '한국철도공사','한국정보통신','농촌진흥청','한국원자력연구원','한국원자력의학원','한국산업기술원',
	    '고등기술연구원','한국인터넷진흥원','한국공항공사','한국식품연구원','한국철도기술연구원',
	    '한국전기연구원','한국핵융합에너지연구원'],
    '국내대학': ['가톨릭대학교', '서울대학교', '고려대학교', '연세대학교', '한국과학기술원','카이스트','kaist', 
        '포항공과대학교','포스텍','postech', '한양대학교', '성균관대학교', '경희대학교','세종대학교',
	    '한국공학대학교','전북대학교','중앙대학교','인하대학교','울산과학기술원','경북대학교','광운대학교',
	    '가천대학교','배재대학교','국민대학교','전남대학교','부산대학교','건국대학교','광주과학기술원',
	    '아주대학교','대구경북과학기술원','충북대학교','충남대학교','동국대학교','한국기술교육대학교',
	    '군산대학교','서울과학기술대학교','숭실대학교','서강대학교','인제대학교','인천대학교','강원대학교',
	    '이화여자대학교','금오공과대학교','한림대학교','동서대학교','영남대학교','부경대학교','계명대학교',
	    '동의대학교','울산대학교','극동대학교','서울시립대학교','단국대학교','경상국립대학교'],
    '해외출원인': ['타이완세미컨덕터', 'taiwansemiconductor','tsmc', '캐논','canon', '애플','apple', '후아웨이','화웨이','huawei', 
	    '인텔','intel', '어플라이드머티어리얼스','어플라이드머티리얼스', 'appliedmaterials', '도요타','토요타','toyota', '한도오따이에네루기',
        '고베세이코쇼', '도쿄엘렉트론','도쿄일렉트론','tokyoelectron', '퀄컴','qualcomm', '마이크론','micron', 
	    '도레이','toray', '후지','fuji', '도시바','toshiba','닛토덴코','스냅','구글','다이니폰인사츠','쌩-고벵글래스프랑스',
	    '제이에프이스틸','아사히가세이','코쿠사이엘렉트릭','텐센트','바이두','다이킨고교',
	    '보에테크놀로지','광동오포','카와사키','텔레호낙티에볼라게트','에이알엠','로베르트보쉬게엠베하','다이셀',
	    '신에쓰가가꾸고교','소니','다이후쿠','스미또모가가꾸','메르크파텐트게엠베하','바스프에스이',
	    '레르리키드쏘시에떼아노님뿌르레뜌드에렉스뿔라따시옹데프로세데조르즈클로드','닛폰세이테츠','다우글로벌테크놀로지스',
	    '닛폰세이테츠','다우글로벌테크놀로지스','미쓰비시','미쯔비시','코니카미놀타','에이에스엠엘', 'asml','코닝','비와이디',
	    '시아오미','샤오미','신에쯔한도타이','세키스이가가쿠고교','가부시끼가이샤레조낙','지멘스','로레알','가부시기가이샤',
	    '마이크로소프트','임머숀','닛산가가쿠','램리써치','닛뽄가야쿠','베이징바이트댄스네트워크','에이지씨','린텍','쿠팡',
	    '오스템','웨이모','레이스트라티직','미츠비시','쓰리엠','샤프','무라타','알까뗄루슨트','히다치','루미리즈','다이요유덴',
	    '에스엠시','샌디스크','미쯔비시','추가이세이야쿠','콘비다와이어리스','앰코','스카이워크스','다이이찌산쿄',
	    '우시오덴키','에프.호프만-라로슈아게','도오꾜오까고오교','사빅글로벌','테슬라','에어프로덕츠앤드케미칼스',
	    '쯔바기모도체인','루미레즈','케이엘에이','베리안','스크린','인터디지털브이씨','테슬라','사우디아라비안오일',
	    '노파르티스아게','옴야인터내셔널아게','베이징다지아인터넷인포메이션','톰슨라이센싱','인터디지털씨이페이튼트에스에이에스',
	    '브리스톨-마이어스스큅','페트로리움앤드케미컬','엔제루구루푸','인터디지탈','유니버셜디스플레이','아폴로인텔리전트커넥티비티',
	    '엔체인','니프코','생-고뱅퍼포먼스플라스틱스','덴소','쟈트코','제이에스알','오므론','돌비','인피니온','제이엑스','일루미나인',
	    '아서스','히타치','지앙수헨그루이파마슈티컬스','상하이마이크로일렉트로닉스이큅먼트','쿠라레','에바라','후루카와덴키고교',
	    '다이오페이퍼','어플라이드메디컬리소시스','모토로라','제넨테크','닛테츠스테인레스','덴카','야스카와덴키','듀폰','에자이',
	    '아마존','레이아','다이요','타이코','엔지케이인슐레이터','디아이씨','이베이','코나미데지타루','에코랍유에스에이','매직립',
	    '니폰제온','다탕모바일','인터내셔널비지네스머신즈', 'ibm','야마하','르노','다이요잉키세이조','다이헨','아지노모토','아틀라스캅코에어파워남로체벤누트삽',
	    '닛폰쇼쿠바이','호야렌즈타일랜드','스미또모가가꾸','도쿄오카고교','베이징센스타임','코르테바애그리사이언스','어드밴스드마이크로디바이시즈',
	    '알박','칭다오하이어워싱머신','가부시끼가이샤구보다','아르끄마프랑스','닛산지도우샤','제네럴일렉트릭','니혼마이크로닉스',
	    '노키아테크놀로지스','엑시스에이비','상하이센스타임인텔리전트테크놀로지','다우실리콘즈','닛테츠케미컬앤드머티리얼','아데카',
	    '스미토모덴키고교','아익스트론에스이','도판','하마마츠포토닉스','에보니크오퍼레이션즈','지에스유아사','프랙스에어테크놀로지',
	    '데쿠세리아루즈','알리바바','가즈트랑스포르에떼끄니가즈','에어버스헬리콥터스','고마쓰세이사쿠쇼']
}

# 'applicant_sgrp' 열 생성
df['applicant_sgrp'] = 'etc'  # 기본값으로 'others' 설정

for key, items in small_grp.items():
    mask = df['applicant_processed'].apply(lambda x: any(item in x for item in items))
    df.loc[mask, 'applicant_sgrp'] = key

# 'applicant_lgrp' 열 생성
df['applicant_lgrp'] = 'etc'  # 기본값으로 'others' 설정

for key, items in large_grp.items():
    mask = df['applicant_processed'].apply(lambda x: any(item in x for item in items))
    df.loc[mask, 'applicant_lgrp'] = key


In [13]:
from konlpy.tag import Okt
import pandas as pd
import re
from collections import Counter

# KoNLPy 형태소 분석기 초기화
okt = Okt()

# 전체 텍스트 합치기
df['title'] = df['title'].astype(str)
df['summary'] = df['summary'].astype(str)

# 불용어(stopwords) 리스트 정의
stopwords = ['상기', '하기', '따라', '본문', '특허', '청구', '발명', '포함', '이상', '이하', '관련', '형성', '서로', '부로', '간의', '상부', '하부',
             '경우', '기능', '구비', '여부', '위해', '사이', '다른','부분','부와', '일부', '대해', '유닛','형태', '말로', '로서', '가중', '추가',
             '해당', '또한', '모드', '다수', '결합', '부가', '연관', '기적','통한', '부의', '현재', '표면', '이의', '동안', '상세', '포인트',
             '내지', '보조', '상부', '하부', '상의', '물의', '거나', '유사', '전체', '동시', '내용', '타입', '분야', '마련', '삽입', '조물',
             '부착', '완료', '제공', '대한', '연결', '구성', '구성', '사용', '상태', '저장', '기초', '전자', '공급', '특징', '디바이스','선택', '획득',
             '효과', '향상', '지지', '물질', '대상', '분리', '수단', '적용', '제거', '접촉', '측면', '기기', '특성', '확인', '단부', '유입',
             '미리', '방식', '대하', '변화', '개구', '관통', '제품', '성능', '생산', '구조체', '활용', '조정', '활용', '메인', '체결', '변경', '절연',
             '크기', '도어', '목적', '코일', '기재', '주변', '안정', '상황', '두께', '여기', '강도', '목표', '단자', '상이', '존재', '립체', '인접',
             '증가', '과제', '최소', '더욱', '따라서', '연속', '일정', '등록', '상호', '지정', '오일', '비용', '중간', '시점', '일체', '일단',
             '이드', '상단', '입구', '물체', '방법', '장치', '포함', '단계', '하나', '기반','실시', '통해','로부터','개시', '각각', '별로', 
             '명세서', '범위', '종류', '가능']


# 텍스트 전처리 및 토큰화 함수 정의
def preprocess_and_tokenize(text):
    text = re.sub(r'[^ ㄱ-ㅣ가-힣]+', '', text)  # 한글만 남기고 모든 문자 제거
    nouns = okt.nouns(text)  # 형태소 분석 후 명사 추출
    nouns = [word for word in nouns if len(word) > 1 and word not in stopwords]  # 한 글자 이상의 명사 중 불용어 제외
    return nouns

# 각 행별로 키워드 추출 및 누적
all_keywords = []
for index, row in df.iterrows():
    text = row['title'] + ' ' + row['summary']
    tokens = preprocess_and_tokenize(text)
    all_keywords.extend(tokens)

# 전체 텍스트에서 빈도 계산 및 상위 빈출 키워드 추출
word_freq = Counter(all_keywords)
top_keywords = word_freq.most_common(1000)

# 결과 출력
#print(top_keywords)

# 출력
for keyword, count in top_keywords:
    print(f"{keyword}: {count} 건")

# 결과를 저장할 파일 경로
output_file = 'keyword_counts_1000.txt'

# 결과 출력 및 파일 저장
with open(output_file, 'w', encoding='utf-8') as f:
    f.write('키워드 건수\n')
    for keyword, count in top_keywords:
        line = f"{keyword}: {count} 건\n"
        f.write(line)

print(f"결과가 {output_file}에 저장되었습니다.")


정보: 248057 건
시스템: 216865 건
데이터: 191395 건
제어: 170431 건
이용: 149481 건
차량: 133264 건
복수: 126736 건
사용자: 119297 건
위치: 110455 건
제조: 105017 건
신호: 99239 건
방향: 93024 건
처리: 92276 건
영역: 88420 건
이미지: 82135 건
모듈: 81336 건
입력: 80569 건
수신: 78514 건
영상: 78147 건
회전: 75442 건
배치: 74742 건
센서: 73434 건
설치: 71133 건
결정: 68139 건
출력: 65680 건
수행: 64629 건
가스: 63188 건
측정: 61254 건
내부: 61160 건
부재: 60684 건
통신: 60195 건
구조: 58346 건
발생: 58290 건
동작: 55159 건
설정: 54478 건
표시: 54416 건
학습: 53111 건
모델: 52460 건
대응: 52121 건
서버: 51823 건
분석: 51644 건
이동: 50967 건
관리: 50298 건
전송: 48217 건
디스플레이: 46704 건
고정: 46544 건
프레임: 45798 건
회로: 45350 건
서비스: 43987 건
외부: 43593 건
조성: 42699 건
금속: 42242 건
공간: 41832 건
예측: 41699 건
전극: 41363 건
추출: 39233 건
단말: 39217 건
전력: 39185 건
온도: 38666 건
감지: 38447 건
시간: 37765 건
방지: 37762 건
반도체: 37382 건
연료: 36644 건
인공: 36541 건
네트워크: 36508 건
판단: 36347 건
화합물: 36169 건
배터리: 35465 건
메모리: 35253 건
블록: 35175 건
지능: 34909 건
전압: 34854 건
주행: 34826 건
공정: 33997 건
결과: 33954 건
프로세서: 32779 건
무선: 32638 건
인식: 32467 건
구동: 31520 건
전달: 31319 건
밸

In [14]:
# top_keywords = word_freq.most_common(1000)

# # 결과 출력
# #print(top_keywords)

In [15]:
# # 출력
# for keyword, count in top_keywords:
#     print(f"{keyword}: {count} 건")


# # 결과를 저장할 파일 경로
# output_file = 'keyword_counts.txt'

# # 결과 출력 및 파일 저장
# with open(output_file, 'w', encoding='utf-8') as f:
#     f.write('키워드 건수\n')
#     for keyword, count in top_keywords:
#         line = f"{keyword}: {count} 건\n"
#         f.write(line)

# print(f"결과가 {output_file}에 저장되었습니다.")

In [48]:
import re

# 각 카테고리와 해당 정규표현식 패턴 정의
categories = {
    '제너럴': r'(인공 지능|인공지능|추론|알고리즘|학습|훈련방법|훈련 방법|지도방법|지도 방법|학습방법|학습 방법|머신러닝|머신 러닝|\
    딥러닝|딥 러닝|패턴인식|패턴 인식|ai|블록체인|블록 체인|이미지인식|이미지 인식|비전|자연어 처리|자연어처리|챗봇|기계학습|기계 학습|심층학습|\
    심층 학습|제스처 인식|제스쳐 인식|피사체 인식|이미지 인식|동작 인식|모션 인식|안면 인식|거대 언어|거대언어|llm|생성형|\
    agi|일반 인공지능|일반 인공 지능|일반 ai|사람 수준|자율 인공지능|자율 ai|자율 학습)',
    '의료': r'(의료|헬스|치료|건강|유전자|병원|환자|의료기기|의료 기기|의료정보|의료 정보|바이오의료|바이오 의료|\
    의학|약물|수술|재활|임상|의사|간호|응급|진단|면역|질환|질병|치료제|보건|헬스케어|생체)',
    '전자상거래서비스': r'(상거래|상품|배송|판매|결제|공급망|공급 망|판매망|판매 망|서비스|전자상거래플랫폼|\
    전자 상거래 플랫폼|전자상거래 플랫폼|온라인쇼핑|온라인 쇼핑|온라인 거래|디지털결제|디지털 결제|\
    온라인서비스|온라인 서비스|온라인 플랫폼|온라인플랫폼|인터넷 거래|인터넷거래|인터넷 서비스|인터넷서비스|\
    고객|구매자|트래픽|방문자|페이지뷰|세션 시간|세션시간|이메일 구독|이메일구독|마케팅|광고|QR|연관 상품|연관상품|\
    통관|관세|환율)',
    '자동차': r'(자동차|주행|자율주행|자율 주행|도로|차량|교통|보행자|차선|신호등|\
    전기차|하이브리드|내연기관|커넥티드카|커넥티드 카|스마트카|스마트 카|운전 보조 시스템|\
    운전보조 시스템|운전보조시스템|ADAS|차량용센서|차량센서|차량 센서|차량용 센서|\
    모빌리티|운송|운전|주차|헤드업 디스플레이|헤드업디스플레이|내비게이션)',
    '금융': r'(금융|코인|트레이딩|투자|신용|자산|블록체인|블록 체인|디지털자산|디지털 자산|\
    간편결제|자동결제|입금|출금|재정|은행|보험|경제|자산|부채|대출|이자|지불|결제|송금|환율|외환|주식|채권|\
    증권|펀드|연금|퇴직금|저축|예금|지급 보증|핀테크|온라인 뱅킹|온라인뱅킹|인터넷 뱅킹|인터넷뱅킹|\
    크라우드 펀딩|크라우드펀딩|크립토커런시|비트코인|알트코인|캐피털)',
    '교육': r'(교육|강의|교육용|시험|성적|학생|학습관리|학습 관리|\
    자기주도학습|학교|대학|강의|교수|교실|e러닝|이러닝|에듀테크|가상 교실|가상 수업|\
    디지털 교과서|교사|교육자)',
    '농업': r'(농업|농사|작황|작물|스마트농업|스마트 농업|농작물|농약|수확|스마트팜|품종)',
    '엔터테인먼트': r'(콘텐츠|컨텐츠|증강 현실|증강현실|게임|미디어|사용자인터페이스|가상현실|음성인식|가상현실콘텐츠|\
    엔터테인먼트|영화|드라마|애니메이션|뮤지컬|공연|콘서트|음악|음원|음반|뮤직비디오|뮤직 비디오|라디오|팟캐스트|\
    팟 캐스트|오디오북|오디오 북|웹툰|웹소설|웹 소설|스트리밍|넷플릭스|디즈니|ott|아마존|유튜브|트위치|틱톡|\
    페이스북|인스타그램|전시회)',
    '보안': r'(보안|침입|탐지|데이터보호|데이터 보호|위협예측|위협 예측|인증|암호화|암호|안전기술|\
    위험|방화벽|바이러스|스파이|랜섬웨어|피싱|스팸|ddos|ids|ips|siem|nac|해독|디지털 서명|\
    디지털서명|디지털 인증서|디지털인증서|ssl|tls|vpn|ipsec|토큰|보안 프로토콜|보안프로토콜|\
    위협 방지|위협 분석|위협 대응|위협 완화|위협 모니터링|위협 방어|위협 평가|위협 예측|위협 방어|\
    위협 차단|위협 관리|침해 탐지|침해 방지|침해 분석|침해 대응|침해 완화|침해 모니터링|침해 평가|\
    침해 예측|침해 방어|침해 차단|침해 관리)',
    '자동화시스템': r'(자동화|로봇자동화|로봇 자동화|제어시스템|제어 시스템|IoT|로봇제어|로봇 제어|\
    스마트시스템|스마트 시스템|온라인서비스|온라인 서비스)'
}
# check_keywords 함수 정의
def check_keywords(text, pattern):
    return 1 if re.search(pattern, text, flags=re.IGNORECASE) else 0

# 공백을 제거하고 비교하는 함수 정의
# def check_keywords(text, pattern):
#     # 문자열의 공백 제거
#     text = text.replace(" ", "")
#     return 1 if re.search(pattern, text, flags=re.IGNORECASE) else 0

# 결측치 처리 후 각 카테고리에 대해 title이나 summary에서 키워드를 찾아서 하나의 컬럼에 표시
for category, pattern in categories.items():
    df[category] = df.apply(lambda row: check_keywords((row['title'] if isinstance(row['title'], str) else '') or (row['summary'] if isinstance(row['summary'], str) else ''), pattern), axis=1)
    #df[category] = df.apply(lambda row: check_keywords((row['title'] if isinstance(row['title'], str) else ''), pattern), axis=1)

df


,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary,...,의료,전자상거래서비스,자동차,금융,교육,농업,엔터테인먼트,보안,자동화시스템,ongoing
0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,20200626,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,"본 발명의 실시 예에 따른 시스템은, 자동화 폐기물 수거함의 딥러닝 기반 인공지능 ...",...,0,0,0,0,0,0,0,0,1,1
1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,20170807,강준철,이영규,강소라,NaN,NaN,"본 발명은, 인공지능(AI)에 관한 기술과 온라인 문자열 사전에 관한 기술을 활용하...",...,0,0,0,0,0,0,0,0,0,1
2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,20220203,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,"본 발명은 품질검사를 위한 장치 및 방법에 관한 것으로, 본 발명에 따르면, 종래,...",...,0,0,0,0,0,0,0,0,0,1
3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,20210812,조국환,NaN,NaN,NaN,NaN,본 발명은 광범위하게 활용되는 인터넷기반 부동산 정보를 때와 장소를 불문하고 기록하...,...,0,0,0,0,0,0,0,0,0,1
4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,20230316,주식회사 오콜로르,박상열,NaN,NaN,NaN,본 발명은 PI(Personal Identity) 정보 빅데이터를 구축하는 통하여 ...,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364207,소멸,LTE에 대한 간섭 소거를 이용한 2차 동기화 신호 검출(SECONDARY SYNC...,1020130082032,20130712,브로드콤 코포레이션,특허법인에이아이피,아바고 테크놀로지스 제너럴 아이피 (싱가포르) 피티이 리미티드,20120723.0,NaN,수신 신호 내의 상대적으로 약한 파워의 SSS 시퀀스들이 검출될 수 있도록 하기 위...,...,0,0,0,0,0,0,0,0,0,0
364208,거절,자동차용 워셔액 냄새 차단 제어 방법(Method for removing smell...,1020130081892,20130712,현대자동차주식회사,한라특허법인(유한),NaN,NaN,NaN,"본 발명은 자동차용 워셔액 냄새 차단 제어 방법에 관한 것으로서, 더욱 상세하게는 ...",...,0,0,1,0,0,0,0,0,0,0
364209,거절,HDMI 케이블 연결 장치 및 방법(HDMI CABLE CONNECT APPARAT...,1020130089860,20130729,인텔 코포레이션,양영준,NaN,20100506.0,NaN,"일부 실시예들에서, HDMI 연결 인터페이스는 디바이스가 어플라이언스에 연결되었는지...",...,0,0,0,0,0,0,0,0,0,0
364210,거절,터빈엔진 세척으로부터 폐수를 수집하기 위한 장치 및 방법(DEVICE AND MET...,1020130073074,20130625,"프래트 앤 휘트니 라인 메인터넌스 서비시즈, 인크.",양영준,NaN,20090330.0,NaN,항공기 터빈 엔진의 세척으로부터 발생되는 폐수를 수집하기 위한 장치가 제공된다. 상...,...,0,0,0,0,0,0,0,0,0,0


In [49]:
print(df['title'].isnull().sum())
print(df['summary'].isnull().sum())

0
6


In [18]:
# df['status'].unique()

In [50]:
### 새로운 컬럼 ongoing 추가 (출원 특허가 계속 중 혹은 등록 되었는지 여부)
### 무효, 취하, 취소, 포기, 소멸, 거절

invalid_words = ['무효', '거절', '취소', '취하' '포기', '소멸']

# status 컬럼의 각 행을 순회하면서 invalid_words에 포함된 단어가 있는지 여부를 판단하여 onGoing 컬럼을 생성
df['ongoing'] = df['status'].apply(lambda x: 0 if any(word in x for word in invalid_words) else 1)

# 결과 확인
df

,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary,...,의료,전자상거래서비스,자동차,금융,교육,농업,엔터테인먼트,보안,자동화시스템,ongoing
0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,20200626,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,"본 발명의 실시 예에 따른 시스템은, 자동화 폐기물 수거함의 딥러닝 기반 인공지능 ...",...,0,0,0,0,0,0,0,0,1,1
1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,20170807,강준철,이영규,강소라,NaN,NaN,"본 발명은, 인공지능(AI)에 관한 기술과 온라인 문자열 사전에 관한 기술을 활용하...",...,0,0,0,0,0,0,0,0,0,1
2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,20220203,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,"본 발명은 품질검사를 위한 장치 및 방법에 관한 것으로, 본 발명에 따르면, 종래,...",...,0,0,0,0,0,0,0,0,0,1
3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,20210812,조국환,NaN,NaN,NaN,NaN,본 발명은 광범위하게 활용되는 인터넷기반 부동산 정보를 때와 장소를 불문하고 기록하...,...,0,0,0,0,0,0,0,0,0,1
4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,20230316,주식회사 오콜로르,박상열,NaN,NaN,NaN,본 발명은 PI(Personal Identity) 정보 빅데이터를 구축하는 통하여 ...,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364207,소멸,LTE에 대한 간섭 소거를 이용한 2차 동기화 신호 검출(SECONDARY SYNC...,1020130082032,20130712,브로드콤 코포레이션,특허법인에이아이피,아바고 테크놀로지스 제너럴 아이피 (싱가포르) 피티이 리미티드,20120723.0,NaN,수신 신호 내의 상대적으로 약한 파워의 SSS 시퀀스들이 검출될 수 있도록 하기 위...,...,0,0,0,0,0,0,0,0,0,0
364208,거절,자동차용 워셔액 냄새 차단 제어 방법(Method for removing smell...,1020130081892,20130712,현대자동차주식회사,한라특허법인(유한),NaN,NaN,NaN,"본 발명은 자동차용 워셔액 냄새 차단 제어 방법에 관한 것으로서, 더욱 상세하게는 ...",...,0,0,1,0,0,0,0,0,0,0
364209,거절,HDMI 케이블 연결 장치 및 방법(HDMI CABLE CONNECT APPARAT...,1020130089860,20130729,인텔 코포레이션,양영준,NaN,20100506.0,NaN,"일부 실시예들에서, HDMI 연결 인터페이스는 디바이스가 어플라이언스에 연결되었는지...",...,0,0,0,0,0,0,0,0,0,0
364210,거절,터빈엔진 세척으로부터 폐수를 수집하기 위한 장치 및 방법(DEVICE AND MET...,1020130073074,20130625,"프래트 앤 휘트니 라인 메인터넌스 서비시즈, 인크.",양영준,NaN,20090330.0,NaN,항공기 터빈 엔진의 세척으로부터 발생되는 폐수를 수집하기 위한 장치가 제공된다. 상...,...,0,0,0,0,0,0,0,0,0,0


In [52]:
#df.to_csv('result_28June2024.csv', encoding='utf-8-sig',index=False)
df.to_excel('result_28June2024.xlsx')
#df=pd.read_excel('result_28June2024.xlsx')

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl


df_mod=pd.read_excel('result_28June2024.xlsx')

In [54]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364212 entries, 0 to 364211
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Unnamed: 0                 364212 non-null  int64  
 1   status                     364212 non-null  object 
 2   title                      364212 non-null  object 
 3   ap_num                     364212 non-null  int64  
 4   application_date           364212 non-null  int64  
 5   applicant                  364212 non-null  object 
 6   representative             359245 non-null  object 
 7   final_owner                196016 non-null  object 
 8   priority_date              147903 non-null  float64
 9   international_filing_date  101183 non-null  float64
 10  summary                    364206 non-null  object 
 11  applicant_processed        364212 non-null  object 
 12  applicant_sgrp             364212 non-null  object 
 13  applicant_lgrp             36

In [55]:
df_mod.columns

Index(['Unnamed: 0', 'status', 'title', 'ap_num', 'application_date',
       'applicant', 'representative', 'final_owner', 'priority_date',
       'international_filing_date', 'summary', 'applicant_processed',
       'applicant_sgrp', 'applicant_lgrp', '제너럴', '의료', '전자상거래서비스', '자동차',
       '금융', '교육', '농업', '엔터테인먼트', '보안', '자동화시스템', 'ongoing'],
      dtype='object')

In [56]:
column_list = ['제너럴', '의료', '전자상거래서비스', '자동차', '금융', '교육', '농업', '엔터테인먼트', '보안', '자동화시스템']

counts = {}
for col in column_list:
    counts[col] = df_mod[col].sum()

print(counts)

{'제너럴': np.int64(44339), '의료': np.int64(16883), '전자상거래서비스': np.int64(11275), '자동차': np.int64(34379), '금융': np.int64(3503), '교육': np.int64(2121), '농업': np.int64(888), '엔터테인먼트': np.int64(7437), '보안': np.int64(8169), '자동화시스템': np.int64(5871)}


In [57]:
column_list = ['제너럴', '의료', '전자상거래서비스', '자동차', '금융', '교육', '농업', '엔터테인먼트', '보안', '자동화시스템']

# 모든 컬럼들이 0인 행의 개수를 세기 위해 각 행의 값을 모든 컬럼들에 대해 합한 후 그 합이 0인 경우를 센다.
num_rows_all_zero = len(df_mod[(df_mod[column_list].sum(axis=1) == 0)])

print("모든 컬럼들이 0인 행의 개수:", num_rows_all_zero)

모든 컬럼들이 0인 행의 개수: 251320


In [61]:
df_mod['application_date'] = pd.to_datetime(df_mod['application_date'], format='%Y%m%d')

# 'application_year' 컬럼 추가 (년도만 추출하여 넣기)
df_mod['application_year'] = df_mod['application_date'].dt.year
# 연도별 출원 건수를 계산
df_mod.pivot_table(index='application_year', aggfunc='count', fill_value=0)

,Unnamed: 0,ap_num,applicant,applicant_lgrp,applicant_processed,applicant_sgrp,application_date,final_owner,international_filing_date,ongoing,...,교육,금융,농업,보안,엔터테인먼트,의료,자동차,자동화시스템,전자상거래서비스,제너럴
application_year,,,,,,,,,,,,,,,,,,,,,
2013,19170,19170,19170,19170,19170,19170,19170,13303,5244,19170,...,19170,19170,19170,19170,19170,19170,19170,19170,19170,19170
2014,32590,32590,32590,32590,32590,32590,32590,22755,10134,32590,...,32590,32590,32590,32590,32590,32590,32590,32590,32590,32590
2015,31971,31971,31971,31971,31971,31971,31971,22810,10449,31971,...,31971,31971,31971,31971,31971,31971,31971,31971,31971,31971
2016,30993,30993,30993,30993,30993,30993,30993,21562,9753,30993,...,30993,30993,30993,30993,30993,30993,30993,30993,30993,30993
2017,31580,31580,31580,31580,31580,31580,31580,23470,10373,31580,...,31580,31580,31580,31580,31580,31580,31580,31580,31580,31580
2018,33146,33146,33146,33146,33146,33146,33146,23355,10287,33146,...,33146,33146,33146,33146,33146,33146,33146,33146,33146,33146
2019,37287,37287,37287,37287,37287,37287,37287,20261,11040,37287,...,37287,37287,37287,37287,37287,37287,37287,37287,37287,37287
2020,42782,42782,42782,42782,42782,42782,42782,18847,12081,42782,...,42782,42782,42782,42782,42782,42782,42782,42782,42782,42782
2021,48085,48085,48085,48085,48085,48085,48085,16314,12310,48085,...,48085,48085,48085,48085,48085,48085,48085,48085,48085,48085


In [59]:
df_mod.head()

,Unnamed: 0,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,...,전자상거래서비스,자동차,금융,교육,농업,엔터테인먼트,보안,자동화시스템,ongoing,application_year
0,0,등록,자동화 폐기물 수거함의 딥러닝 기반 인공지능 운영 시스템 및 방법(OPERATION...,1020200078616,2020-06-26,(주)에이씨아이케미칼아시아,고영갑,(주)에이씨아이케미칼아시아,NaN,NaN,...,0,0,0,0,0,0,0,1,1,2020
1,1,등록,"인공지능(AI)을 통한 딥러닝훈련모듈과, 순위화프레임워크모듈을 활용하여, 법률전문가...",1020170099431,2017-08-07,강준철,이영규,강소라,NaN,NaN,...,0,0,0,0,0,0,0,0,1,2017
2,2,공개,인공지능을 이용한 품질검사 시스템 및 방법(System and method for ...,1020220014151,2022-02-03,모던에이아이비전솔루션 주식회사,김정수,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,2022
3,3,공개,파이썬 딥러닝 기반 인공지능(AI) 부동산 컨설팅 매물장(Python deep le...,1020210106993,2021-08-12,조국환,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,2021
4,4,거절,PI 기반 빅데이터 퍼스널 컬러 진단 인공지능 딥러닝 알고리즘 시스템 운영방법(PI...,1020230034713,2023-03-16,주식회사 오콜로르,박상열,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,2023


In [21]:
# 기술 유형별 키워드 리스트
## 지금은 연도별... 상위출원인에 대해서, 관심가는 출원인에 대해서..
keywords_dict = {
    '머신러닝': ['머신러닝', '기계 학습', '기계학습', 'Machine Learning', 'ML'],
    '딥러닝': ['딥러닝', '심층 학습', '심층학습', 'Deep Learning', 'DL'],
    '인공지능': ['인공지능', 'AI', 'Artificial Intelligence'],
    '컴퓨터 비전': ['컴퓨터 비전', 'Computer Vision', 'CV'],
    '자연어 처리': ['자연어 처리', 'Natural Language Processing', 'NLP'],
    '로봇공학': ['로봇공학', '로봇', 'Robotics'],
    '빅데이터': ['빅데이터', 'Big Data'],
    '사물인터넷': ['사물인터넷', 'IoT', 'Internet of Things'],
    '클라우드 컴퓨팅': ['클라우드 컴퓨팅', 'Cloud Computing'],
    '블록체인': ['블록체인', 'Blockchain'],
    '이미지 모션 인식':['제스처','제스쳐', '피사체','이미지', '동작 인식', '동작인식', '모션 인식','모션인식', '안면'],
    '거대언어':['자연어처리','자연어 처리', '챗봇','거대 언어','거대언어','llm'],
    '생성형': ['생성형','그림 그리기','음성 생성', '보이스', '영상 생성', '영상 제작', '음악 제작','사진 생성', '글쓰기', '이미지 생성'],
    '일반 인공지능':['agi', '일반 인공지능', '일반 ai', '사람 수준', '자율 인공지능', '자율 AI', '자율 학습']
}

# 연도별로 기술 유형별 키워드가 포함된 타이틀의 개수를 저장할 딕셔너리 초기화
yearly_counts = {year: {tech: 0 for tech in keywords_dict.keys()} for year in range(2013, 2025)}

# 각 연도별로 기술 유형별 키워드가 포함된 타이틀의 개수를 계산
for index, row in df.iterrows():
    title = row['title']  ########################################### 여기에 요악도 포함
    application_date = row['application_date']
    if pd.notna(title) and pd.notna(application_date):
        year = int(str(application_date)[:4])  # 날짜의 앞 4자리를 연도로 추출
        if 2013 <= year <= 2024:
            for tech, keywords in keywords_dict.items():
                if any(keyword.lower() in title.lower() for keyword in keywords):
                    yearly_counts[year][tech] += 1
                    break  # 한 번 카운트되면 다른 키워드는 확인하지 않음

# 결과 출력
for year, counts in yearly_counts.items():
    print(f"Year: {year}")
    for tech, count in counts.items():
        print(f"  {tech}: {count}")

Year: 2013
  머신러닝: 15
  딥러닝: 168
  인공지능: 1630
  컴퓨터 비전: 20
  자연어 처리: 0
  로봇공학: 85
  빅데이터: 3
  사물인터넷: 16
  클라우드 컴퓨팅: 2
  블록체인: 0
  이미지 모션 인식: 129
  거대언어: 1
  생성형: 6
  일반 인공지능: 193
Year: 2014
  머신러닝: 23
  딥러닝: 306
  인공지능: 2548
  컴퓨터 비전: 23
  자연어 처리: 1
  로봇공학: 148
  빅데이터: 14
  사물인터넷: 39
  클라우드 컴퓨팅: 2
  블록체인: 0
  이미지 모션 인식: 236
  거대언어: 2
  생성형: 12
  일반 인공지능: 406
Year: 2015
  머신러닝: 34
  딥러닝: 303
  인공지능: 2488
  컴퓨터 비전: 31
  자연어 처리: 2
  로봇공학: 152
  빅데이터: 11
  사물인터넷: 44
  클라우드 컴퓨팅: 5
  블록체인: 0
  이미지 모션 인식: 238
  거대언어: 3
  생성형: 12
  일반 인공지능: 396
Year: 2016
  머신러닝: 52
  딥러닝: 283
  인공지능: 2490
  컴퓨터 비전: 10
  자연어 처리: 0
  로봇공학: 190
  빅데이터: 18
  사물인터넷: 74
  클라우드 컴퓨팅: 5
  블록체인: 0
  이미지 모션 인식: 286
  거대언어: 3
  생성형: 17
  일반 인공지능: 375
Year: 2017
  머신러닝: 96
  딥러닝: 388
  인공지능: 2746
  컴퓨터 비전: 20
  자연어 처리: 3
  로봇공학: 216
  빅데이터: 22
  사물인터넷: 108
  클라우드 컴퓨팅: 1
  블록체인: 0
  이미지 모션 인식: 219
  거대언어: 27
  생성형: 15
  일반 인공지능: 402
Year: 2018
  머신러닝: 159
  딥러닝: 561
  인공지능: 3392
  컴퓨터 비전: 36
  자연어 처리: 6
  로봇공학: 296
  빅데이터:

몇몇 유명한 인공지능 관련 벤처/스타트업의 출원 건수를 한번 보자

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl


df=pd.read_excel('patents_data_test.xlsx')

In [63]:
df

,Unnamed: 0,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,summary
0,193,거절,딥러닝 연산에서의 인-메모리 그리드 운용 방법 및 장치(Method and devi...,1020190125743,2019-10-11,주식회사 시스기어,정승훈,NaN,NaN,NaN,"본 발명은 인메모리 기반 딥러닝 연산을 위한 장치 및 방법에 관한 것으로, 인공지능..."
1,241,등록,두피상태 정보를 기반으로 한 두피유형 진단 시스템 및 이를 이용한 두피 개선 방법(...,1020210148436,2021-11-02,아람휴비스 주식회사,서동원,아람휴비스 주식회사,NaN,NaN,본 발명은 두피상태 정보를 기반으로 한 두피유형 진단 시스템 및 이를 이용한 두피 ...
2,324,등록,고해상도 드론 동영상과 딥러닝 객체 인식기술을 활용한 실시간 습지 지역 주요 식생 ...,1020220143895,2022-11-01,(주) 지오씨엔아이,김성대,(주) 지오씨엔아이,NaN,NaN,본 발명은 본 발명은 고해상도 드론 동영상과 딥러닝 객체 인식기술을 활용한 실시간 ...
3,544,등록,딥러닝 기술과 이미지 특징 추출 기술을 결합한 영상물 내 미술품 인식 방법(Meth...,1020180009129,2018-01-25,상명대학교산학협력단,특허법인세원,이기철,NaN,NaN,본 발명은 컴퓨터 장치에 의해 구현되는 영상물 내의 미술품을 인식하는 시스템과 방법...
4,672,등록,딥러닝 기반의 요약문 생성 정확도 개선 시스템(SYSTEM FOR IMPROVING...,1020230104375,2023-08-09,(주)유알피,김창덕,(주)유알피,NaN,NaN,"본 발명은 딥러닝 기반의 요약문 생성 정확도 개선 시스템에 관한 것으로, 딥러닝 기..."
5,734,공개,영상정보를 이용한 외부환경정보 추정시스템 및 방법(System and method ...,1020220048337,2022-04-19,한국해양과학기술원,김정수,NaN,NaN,NaN,"본 발명은 외부환경정보를 측정하기 위한 장치 및 방법에 관한 것으로, 본 발명에 따..."


In [64]:
# 필터링할 키워드 설정
keywords = ['인공 지능','인공지능','추론','신경망', '신경 망','학습','훈련방법','훈련 방법','지도방법','지도 방법','학습방법','학습 방법','머신러닝','머신 러닝',\
    '딥러닝','딥 러닝','패턴인식','패턴 인식','블록체인','블록 체인','이미지인식','이미지 인식','비전','자연어 처리','자연어처리','챗봇','기계학습','기계 학습','심층학습',\
    '심층 학습','제스처 인식','제스쳐 인식','피사체 인식','동작 인식','모션 인식','안면 인식','거대 언어','거대언어','생성형 인공지능',\
    '생성형 ai','일반 인공지능','일반 인공 지능','일반 ai','자율 인공지능','자율 ai','자율 학습']

# 'title' 컬럼 또는 'summary' 컬럼에 키워드가 포함된 경우만 남기기
filtered_df = df[df['title'].str.contains('|'.join(keywords), case=False, na=False) |
                 df['summary'].str.contains('|'.join(keywords), case=False, na=False)]

# 필터링된 행의 인덱스를 가져오기
filtered_index = filtered_df.index

# 원본 데이터프레임에서 필터링된 인덱스에 해당하는 행만 남기기
fdf = df.loc[filtered_index]

# 결과 출력
#print(fdf)
print(df.shape, fdf.shape)

(6, 11) (6, 11)


In [66]:
import re

# 각 카테고리와 해당 정규표현식 패턴 정의
categories = {
    '제너럴': r'(인공 지능|인공지능|추론|신경망|신경 망|학습|훈련방법|훈련 방법|지도방법|지도 방법|학습방법|학습 방법|머신러닝|머신 러닝|딥러닝|딥 러닝|패턴인식|패턴 인식|블록체인|블록 체인|이미지인식|이미지 인식|비전|자연어 처리|자연어처리|챗봇|기계학습|기계 학습|심층학습|심층 학습|제스처 인식|제스쳐 인식|피사체 인식|동작 인식|모션 인식|안면 인식|거대 언어|거대언어|생성형|일반 인공지능|일반 인공 지능|일반 ai|자율 인공지능|자율 ai|자율 학습)',
    '의료': r'(의료|헬스|치료|건강|유전자|병원|환자|의료기기|의료 기기|의료정보|의료 정보|바이오의료|바이오 의료|
    의학|약물|수술|재활|임상|의사|간호|응급|진단|면역|질환|질병|치료제|보건|헬스케어|생체)',
    '전자상거래서비스': r'(상거래|상품|배송|판매|결제|공급망|공급 망|판매망|판매 망|서비스|전자상거래플랫폼|\
    전자 상거래 플랫폼|전자상거래 플랫폼|온라인쇼핑|온라인 쇼핑|온라인 거래|디지털결제|디지털 결제|\
    온라인서비스|온라인 서비스|온라인 플랫폼|온라인플랫폼|인터넷 거래|인터넷거래|인터넷 서비스|인터넷서비스|\
    고객|구매자|트래픽|방문자|페이지뷰|세션 시간|세션시간|이메일 구독|이메일구독|마케팅|광고|QR|연관 상품|연관상품|\
    통관|관세|환율)',
    '자동차': r'(자동차|주행|자율주행|자율 주행|도로|차량|교통|보행자|차선|신호등|\
    전기차|하이브리드|내연기관|커넥티드카|커넥티드 카|스마트카|스마트 카|운전 보조 시스템|\
    운전보조 시스템|운전보조시스템|ADAS|차량용센서|차량센서|차량 센서|차량용 센서|\
    모빌리티|운송|운전|주차|헤드업 디스플레이|헤드업디스플레이|내비게이션)',
    '금융': r'(금융|코인|트레이딩|투자|신용|자산|블록체인|블록 체인|디지털자산|디지털 자산|\
    간편결제|자동결제|입금|출금|재정|은행|보험|경제|자산|부채|대출|이자|지불|결제|송금|환율|외환|주식|채권|\
    증권|펀드|연금|퇴직금|저축|예금|지급 보증|핀테크|온라인 뱅킹|온라인뱅킹|인터넷 뱅킹|인터넷뱅킹|\
    크라우드 펀딩|크라우드펀딩|크립토커런시|비트코인|알트코인|캐피털)',
    '교육': r'(교육|강의|교육용|시험|성적|학생|학습관리|학습 관리|\
    자기주도학습|학교|대학|강의|교수|교실|e러닝|이러닝|에듀테크|가상 교실|가상 수업|\
    디지털 교과서|교사|교육자)',
    '농업': r'(농업|농사|작황|작물|스마트농업|스마트 농업|농작물|농약|수확|스마트팜|품종)',
    '엔터테인먼트': r'(콘텐츠|컨텐츠|증강 현실|증강현실|게임|미디어|사용자인터페이스|가상현실|음성인식|가상현실콘텐츠|\
    엔터테인먼트|영화|드라마|애니메이션|뮤지컬|공연|콘서트|음악|음원|음반|뮤직비디오|뮤직 비디오|라디오|팟캐스트|\
    팟 캐스트|오디오북|오디오 북|웹툰|웹소설|웹 소설|스트리밍|넷플릭스|디즈니|ott|아마존|유튜브|트위치|틱톡|\
    페이스북|인스타그램|전시회)',
    '보안': r'(보안|침입|탐지|데이터보호|데이터 보호|위협예측|위협 예측|인증|암호화|암호|안전기술|\
    위험|방화벽|바이러스|스파이|랜섬웨어|피싱|스팸|ddos|ids|ips|siem|nac|해독|디지털 서명|\
    디지털서명|디지털 인증서|디지털인증서|ssl|tls|vpn|ipsec|토큰|보안 프로토콜|보안프로토콜|\
    위협 방지|위협 분석|위협 대응|위협 완화|위협 모니터링|위협 방어|위협 평가|위협 예측|위협 방어|\
    위협 차단|위협 관리|침해 탐지|침해 방지|침해 분석|침해 대응|침해 완화|침해 모니터링|침해 평가|\
    침해 예측|침해 방어|침해 차단|침해 관리)',
    '자동화시스템': r'(자동화|로봇자동화|로봇 자동화|제어시스템|제어 시스템|IoT|로봇제어|로봇 제어|\
    스마트시스템|스마트 시스템|온라인서비스|온라인 서비스)'
}
# check_keywords 함수 정의
def check_keywords(text, pattern):
    return 1 if re.search(pattern, text, flags=re.IGNORECASE) else 0

# 공백을 제거하고 비교하는 함수 정의
# def check_keywords(text, pattern):
#     # 문자열의 공백 제거
#     text = text.replace(" ", "")
#     return 1 if re.search(pattern, text, flags=re.IGNORECASE) else 0

# 결측치 처리 후 각 카테고리에 대해 title이나 summary에서 키워드를 찾아서 하나의 컬럼에 표시
for category, pattern in categories.items():
    fdf[category] = fdf.apply(lambda row: check_keywords((row['title'] if isinstance(row['title'], str) else '') or (row['summary'] if isinstance(row['summary'], str) else ''), pattern), axis=1)
    #df[category] = df.apply(lambda row: check_keywords((row['title'] if isinstance(row['title'], str) else ''), pattern), axis=1)

fdf.head()


,Unnamed: 0,status,title,ap_num,application_date,applicant,representative,final_owner,priority_date,international_filing_date,...,제너럴,의료,전자상거래서비스,자동차,금융,교육,농업,엔터테인먼트,보안,자동화시스템
0,193,거절,딥러닝 연산에서의 인-메모리 그리드 운용 방법 및 장치(Method and devi...,1020190125743,2019-10-11,주식회사 시스기어,정승훈,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
1,241,등록,두피상태 정보를 기반으로 한 두피유형 진단 시스템 및 이를 이용한 두피 개선 방법(...,1020210148436,2021-11-02,아람휴비스 주식회사,서동원,아람휴비스 주식회사,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
2,324,등록,고해상도 드론 동영상과 딥러닝 객체 인식기술을 활용한 실시간 습지 지역 주요 식생 ...,1020220143895,2022-11-01,(주) 지오씨엔아이,김성대,(주) 지오씨엔아이,NaN,NaN,...,1,0,0,0,0,0,0,0,1,0
3,544,등록,딥러닝 기술과 이미지 특징 추출 기술을 결합한 영상물 내 미술품 인식 방법(Meth...,1020180009129,2018-01-25,상명대학교산학협력단,특허법인세원,이기철,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
4,672,등록,딥러닝 기반의 요약문 생성 정확도 개선 시스템(SYSTEM FOR IMPROVING...,1020230104375,2023-08-09,(주)유알피,김창덕,(주)유알피,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
